In [ ]:
# The standard Python science stack
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.sparse.linalg import lsqr

# Plotting maps using GMT
import pygmt

# The Fatiando stack
import harmonica as hm
from grav_column_der import grav_column_der

# RIS tight extent
# e_fig = -560000
# w_fig = 400000
# n_fig = -1400000
# s_fig = -440000

# RIS zoomed in extent
zoom = 240e3
e_fig = -560000+zoom
w_fig = 400000-zoom
n_fig = -1400000+zoom
s_fig = -440000-zoom
fig_reg = [e_fig, w_fig, n_fig, s_fig]

fig_height = 80
fig_width = fig_height*(w_fig-e_fig)/(s_fig-n_fig)
fig_ratio = (s_fig-n_fig)/(fig_height/1000)
fig_proj = f"x1:{fig_ratio}"

buffer = 400e3

e_inv = int(e_fig-buffer)
w_inv = int(w_fig+buffer)
n_inv = int(n_fig-buffer)
s_inv = int(s_fig+buffer)
inv_reg = [e_inv, w_inv, n_inv, s_inv]
inv_reg_str=f'{e_inv}/{w_inv}/{n_inv}/{s_inv}'

inv_height = 80
inv_width = inv_height*(w_inv-e_inv)/(s_inv-n_inv)
inv_ratio = (s_inv-n_inv)/(inv_height/1000)
inv_proj = f"x1:{inv_ratio}"

# Import data

In [ ]:
inv_reg_str=f'{e_inv}/{w_inv}/{n_inv}/{s_inv}'

layers_list =[
             'ice',
             'water',
             'layer1',
            #  'layer2',
]
spacing_list = [
                20e3,
                20e3,
                20e3,
                # 40e3,
]
rho_list = [
            920,
            1030,
            2600,
            # 2800,
]

grav_spacing = 20e3

# Bedmachine grids
!gmt grdsample -R{inv_reg_str} -rg -Vn -I{grav_spacing}  ../gravity_data/ant4d_gravity.nc -G../inversion_layers/gravity_inv.nc
!gmt grdsample -R{inv_reg_str} -rg -Vn -I{spacing_list[0]}  ../inversion_layers/bedmachine/BedMachine_surface_5k_filled.nc -G../inversion_layers/BedMachine_surface_filled_inv.nc
!gmt grdsample -R{inv_reg_str} -rg -Vn -I{spacing_list[1]}  ../inversion_layers/bedmachine/BedMachine_icebase_5k_filled.nc -G../inversion_layers/BedMachine_icebase_filled_inv.nc
!gmt grdsample -R{inv_reg_str} -rg -Vn -I{spacing_list[2]}  ../inversion_layers/bedmachine/BedMachine_bed_5k.nc -G../inversion_layers/BedMachine_bed_inv.nc 
# !gmt grdsample -R{inv_reg_str} -rg -Vn -I{spacing_list[3]}  ../inversion_layers/RE_basement_BedMachinebed_filled.nc -G../inversion_layers/ROSETTA_basement_BedMachine_bed_inv.nc 
# !gmt grdsample -R{inv_reg_str} -rg -Vn -I{spacing_list[3]}  ../inversion_layers/ANTASed_mag_basement_filled.nc -G../inversion_layers/ROSETTA_basement_BedMachine_bed_inv.nc 

# Fabio's test grids
# inv_reg_str=f'{e_inv}/{w_inv}/{n_inv}/{s_inv}'
# !gmt grdsample -R{inv_reg_str} -rg -I10e3  inversion_layers/base_RIS_gravity_10000m_lr.nc -Ginversion_layers/gravity_inv.nc
# !gmt grdsample -R{inv_reg_str} -rg -I10e3  inversion_layers/base_RIS_ice_10000m_lr.nc -Ginversion_layers/BedMachine_surface_filled_inv.nc
# !gmt grdsample -R{inv_reg_str} -rg -I10e3  inversion_layers/base_RIS_water_10000m_lr.nc -Ginversion_layers/BedMachine_icebase_filled_inv.nc
# !gmt grdsample -R{inv_reg_str} -rg -I10e3  inversion_layers/base_RIS_layer1_10000m_lr.nc -Ginversion_layers/BedMachine_bed_inv.nc 
# !gmt grdsample -R{inv_reg_str} -rg -I20e3  inversion_layers/base_RIS_layer2_20000m_lr.nc -Ginversion_layers/ROSETTA_basement_BedMachine_bed_inv.nc 

fname_list=[
            'C:/Users/matthewt/Documents/Python_Scripts/RIS_gravity_inversion/inversion_layers/BedMachine_surface_filled_inv.nc',
            'C:/Users/matthewt/Documents/Python_Scripts/RIS_gravity_inversion/inversion_layers/BedMachine_icebase_filled_inv.nc',
            'C:/Users/matthewt/Documents/Python_Scripts/RIS_gravity_inversion/inversion_layers/BedMachine_bed_inv.nc',
            # 'C:/Users/matthewt/Documents/Python_Scripts/RIS_gravity_inversion/inversion_layers/ROSETTA_basement_BedMachine_bed_inv.nc',
            ]

layers = {j:{'spacing':spacing_list[i], 
            'fname':fname_list[i], 
            'rho':rho_list[i]} for i, j in enumerate(layers_list)}

for k, v in layers.items():
    v['grid']=xr.load_dataset(v['fname'])
    v['df']=xr.load_dataset(v['fname']).to_dataframe().reset_index()
    v['df']['rho']=v['rho']
    v['df'].dropna(how='any', inplace=True)
    v['len']=len(v['df'].x)  

fname='C:/Users/matthewt/Documents/Python_Scripts/RIS_gravity_inversion/inversion_layers/gravity_inv.nc'
grid_grav = xr.load_dataset(fname)           
df_grav = xr.load_dataset(fname).to_dataframe().reset_index() 
df_grav.rename(columns={'z':'Gobs'}, inplace=True)
df_grav.dropna(how='any', inplace=True)
df_grav['z']=layers['ice']['grid'].to_dataframe().reset_index().z+750

for k, v in layers.items():
    print(f"points in {k} file: {v['len']}") 

print(f'points in gravity file: {len(df_grav)}')    

# Show layers

In [ ]:
fig = pygmt.Figure()

pygmt.grd2cpt(cmap="jet", grid=grid_grav.z, region=fig_reg, background=True)
fig.grdimage(
    grid_grav.z,
    cmap=True,
    projection=inv_proj, 
    region=inv_reg,
    nan_transparent=True,
    frame=['+gwhite'])
fig.colorbar(cmap=True, position='jBC+jTC+h', frame='x+l"observed gravity (mGal)"')
# plot later figure extents (without inv buffer)
fig.plot(x = [e_fig, e_fig, w_fig, w_fig, e_fig], 
             y = [n_fig, s_fig, s_fig, n_fig, n_fig], 
             pen = '2p,black')

fig.shift_origin(xshift=(inv_width+2)/10)

for i, (k, v) in enumerate(layers.items()):
    data=layers[k]['grid'].z
    fig.grdimage(
        data,
        cmap='globe',
        projection=inv_proj, 
        region=inv_reg,
        nan_transparent=True,
        frame=['+gwhite'])
    fig.colorbar(cmap='globe', position='jBC+jTC+h', frame=f'x+l"{k} elevation (m)"')

    # plot later figure extents (without inv buffer)
    fig.plot(x = [e_fig, e_fig, w_fig, w_fig, e_fig], 
             y = [n_fig, s_fig, s_fig, n_fig, n_fig], 
             pen = '2p,black')
             
    fig.shift_origin(xshift=(inv_width+2)/10)
    # if i+1 == len(layers)//2:
    #     fig.shift_origin(yshift=(-inv_height-15)/10, xshift=((-inv_width-2)*1.5)/10)
    # else:
    #     fig.shift_origin(xshift=(inv_width+2)/10)
fig.show()

# Forward gravity calculations of layers

In [ ]:
# Calculate inital forward gravity model of input layer
# add density variable to datasets
for k ,v in layers.items():
    v['grid']['density']=v['grid'].z.copy()
    v['grid'].density.values[:] = v['rho']

# list of layers, bottom up
reversed_layers_list = np.flip(layers_list[:])

for i, j in enumerate(reversed_layers_list):
    if i == 0:
        layers[j]['prisms']=hm.prism_layer(
            coordinates=(list(layers[j]['grid'].x), list(layers[j]['grid'].y)),   
            surface=layers[j]['grid'].z, 
            reference=-50e3,
            properties={'density':layers[j]['grid'].density})
        print(f'mean elevation of {j} top: {layers[j]["prisms"].top.values.mean()} and bottom: {layers[j]["prisms"].bottom.values.mean()}')

    else:
        if layers[j]['spacing'] != layers[reversed_layers_list[i-1]]['spacing']:
            print(f"resolutions don't match for {j} ({layers[j]['spacing']}m) and {reversed_layers_list[i-1]} ({layers[reversed_layers_list[i-1]]['spacing']}m)")
            print(f"resampling {reversed_layers_list[i-1]} to match {j}")
            tmp = layers[j]['grid'].z.to_dataframe().reset_index()
            tmp_regrid = pygmt.grdtrack(points = tmp[['x','y']], 
                                        grid = layers[reversed_layers_list[i-1]]['grid'].z, 
                                        newcolname = 'z_regrid')
            tmp['z_low']=tmp.merge(tmp_regrid, how = 'left', on = ['x','y']).z_regrid
            tmp_grd = pygmt.xyz2grd(tmp[['x','y','z_low']], region = inv_reg, spacing = layers[j]['spacing'])

            layers[j]['prisms']=hm.prism_layer(
                coordinates=(list(layers[j]['grid'].x), list(layers[j]['grid'].y)),   
                surface=layers[j]['grid'].z, 
                reference=tmp_grd,
                properties={'density':layers[j]['grid'].density})
            print(f'mean elevation of {j} top: {layers[j]["prisms"].top.values.mean()} and bottom: {layers[j]["prisms"].bottom.values.mean()}')
        else:
            layers[j]['prisms']=hm.prism_layer(
                coordinates=(list(layers[j]['grid'].x), list(layers[j]['grid'].y)),   
                surface=layers[j]['grid'].z, 
                reference=layers[reversed_layers_list[i-1]]['grid'].z,
                properties={'density':layers[j]['grid'].density})
            print(f'mean elevation of {j} top: {layers[j]["prisms"].top.values.mean()} and bottom: {layers[j]["prisms"].bottom.values.mean()}')

print('starting forward gravity calculations')
for k, v in layers.items():
    df_grav[f'{k}_forward_grav'] = v['prisms'].prism_layer.gravity(
        coordinates = (df_grav.x, df_grav.y, df_grav.z),
        field = 'g_z', progressbar = True)
    print(f'finished {k} layer')

# add gravity effects of all input layers
for i, j in enumerate(layers_list):
    if i == 0:
        df_grav['forward_grav_total'] = df_grav[f'{j}_forward_grav']
    else:
        df_grav['forward_grav_total'] += df_grav[f'{j}_forward_grav']

df_grav.describe()


# Plot forward gravity of each layer

In [ ]:
fig = pygmt.Figure()

for i, (k, v) in enumerate(layers.items()):
    data = pygmt.xyz2grd(data=df_grav[['x','y',f'{k}_forward_grav']],
                         region=inv_reg, spacing=grav_spacing, projection=inv_proj) 
    pygmt.grd2cpt(cmap="jet", grid=data, region=fig_reg, background=True, continuous=True)
    fig.grdimage(
        data,
        cmap=True,
        projection=fig_proj, 
        region=fig_reg,
        nan_transparent=True,
        frame=['+gwhite'])
    fig.colorbar(cmap=True, position='jBC+jTC+h', frame=f'x+l"{k} forward gravity (mGal)"')
    
    fig.shift_origin(xshift=(fig_width+2)/10)

    # if i+1 == len(layers)//2:
    #     fig.shift_origin(yshift=(-fig_height-15)/10, xshift=((-fig_width-2)*1)/10)
    # else:
    #     fig.shift_origin(xshift=(fig_width+2)/10)
        
fig.show()

# Plot observed, misfit, and forward gravity 

In [ ]:
# plot observed, forward, and misfit gravity grids

df_grav['initial_misfit'] = df_grav.Gobs - df_grav.forward_grav_total
shift = df_grav.initial_misfit[(df_grav.x.between(e_fig, w_fig)) & (df_grav.y.between(n_fig, s_fig))].mean() 
print(f'average initial misfit within figure region is {int(shift)} mGal')

df_grav['Gobs_shift'] = df_grav.Gobs - shift
print(f'average Gobs_shifted: {int(df_grav.Gobs_shift[(df_grav.x.between(e_fig, w_fig)) & (df_grav.y.between(n_fig, s_fig))].mean())} mGal')

df_grav['forward_grav_misfit'] = df_grav.Gobs_shift - df_grav.forward_grav_total
misfit_shift = df_grav.forward_grav_misfit[(df_grav.x.between(e_fig, w_fig)) & (df_grav.y.between(n_fig, s_fig))].mean() 
print(f'average misfit between Gobs_shift and forward gravity within figure region is {int(misfit_shift)} mGal')

# print(f'average forward gravity total: {int(df_grav.forward_grav_total[(df_grav.x.between(e_fig, w_fig)) & (df_grav.y.between(n_fig, s_fig))].mean())} mGal')

observed_shift_grav_grid=pygmt.xyz2grd(data = df_grav[['x','y','Gobs_shift']],
                                 region=inv_reg, 
                                 spacing=grav_spacing,
                                 projection=inv_proj)

forward_grav_grid=pygmt.xyz2grd(data = df_grav[['x','y','forward_grav_total']],
                                 region=inv_reg, 
                                 spacing=grav_spacing,
                                 projection=inv_proj)

misfit_grav_grid=pygmt.xyz2grd(data = df_grav[['x','y','forward_grav_misfit']],
                                 region=inv_reg, 
                                 spacing=grav_spacing,
                                 projection=inv_proj)


fig = pygmt.Figure()

# plot observed gravity
grid=observed_shift_grav_grid
pygmt.grd2cpt(cmap="jet", grid=grid, region=fig_reg, background=True, continuous=True)
fig.grdimage(
        grid,
        cmap=True,
        projection=fig_proj, 
        region=fig_reg,
        nan_transparent=True,
        frame=['+gwhite'])
fig.colorbar(cmap=True, frame='af+l"observed gravity (mGal)"')

fig.shift_origin(xshift=(fig_width+2)/10)

# plot forward gravity
grid=forward_grav_grid
pygmt.grd2cpt(cmap="jet", grid=grid, region=fig_reg, background=True, continuous=True)
fig.grdimage(
        grid,
        cmap=True,
        projection=fig_proj, 
        region=fig_reg,
        nan_transparent=True,
        frame=['+gwhite'])
fig.colorbar(cmap=True, frame='af+l"forward gravity (mGal)"')


fig.shift_origin(xshift=(fig_width+2)/10)

# plot misfit gravity
grid=misfit_grav_grid
pygmt.grd2cpt(cmap="polar+h0", grid=grid, region=fig_reg, background=True, continuous=True)
fig.grdimage(
        grid,
        cmap=True,#'polar+h0',
        projection=fig_proj, 
        region=fig_reg,
        nan_transparent=True,
        frame=['+gwhite'])
fig.colorbar(cmap=True, frame='af+l"gravity misfit (mGal)"')

fig.show()

"Gobs",df_grav.Gobs.mean(), "Forward",df_grav.forward_grav_total.mean(), "Gobs shift",df_grav.Gobs_shift.mean(), "misfit", df_grav.forward_grav_misfit.mean()

# Prism geometry inversion

In [ ]:
#########
active_layer = 'layer1'
#########

# df = layers[active_layer]['df']
# df['z_updated']=df.z
df = layers[active_layer]['prisms'].to_dataframe().reset_index()
# df['top_updated']=df.top

# df_grav['grav_inversion']=df_grav.forward_grav_misfit
df_grav['grav_inversion']=df_grav.Gobs_shift
# df_grav['grav_inversion']=df_grav.residual_grav 

misfit_sq_tolerance=.01
delta_misfit_squared_tolerance=0.02
Max_Iterations=10
max_layer_change_per_iter=100 

MATDATPAR = (np.zeros([len(df_grav),len(df)])) # array with NG number of rows and NBath+NBase+NM number of columns
MAT_DENS = (np.zeros([len(df_grav),len(df)]))
    
misfit_squared_updated=np.Inf  # positive infinity
delta_misfit_squared=np.Inf  # positive infinity

ITER=0
 # while delta_misfit_squared (inf) is greater than 1 + least squares tolerance (0.02)
while delta_misfit_squared > 1+delta_misfit_squared_tolerance:
    ITER=ITER+1 
    print(f"iteration {ITER}")

    df = layers[active_layer]['prisms'].to_dataframe().reset_index()

    # calculate forward gravity of active layer               
    df_grav[f'forward_grav_{active_layer}_{ITER}'] = layers[active_layer]['prisms'].prism_layer.gravity(
    coordinates=(df_grav.x, df_grav.y, df_grav.z),
    field = 'g_z')
    
    # recalculate misift with new forward grav
    df_grav[f'forward_grav_total_{ITER}'] = df_grav.forward_grav_total - df_grav[f'{active_layer}_forward_grav'] + df_grav[f'forward_grav_{active_layer}_{ITER}']

    df_grav[f'inv_misfit_{ITER}']=df_grav.grav_inversion-df_grav[f'forward_grav_total_{ITER}']
    print(f"gravity misfit before iteration {ITER} = {int(df_grav[f'inv_misfit_{ITER}'].mean())}")

    # calculate MATDATPAR with grav_column_der
    spacing = layers[active_layer]['spacing'] 
    # bottoms=layers[active_layer]['prisms'].to_dataframe().reset_index()
    # df['z_low']=bottoms.bottom[bottoms.top.notna()]
    for i,j in enumerate(tqdm(df_grav.Gobs)):
        # Calculation of the Jacobian matrix using the annular cylinder approximation (grav_column_der)
        #MATDATPAR is matrix array with NG number of rows and NBath+NBase+NM number of columns
        # uses vertical derivative of gravity to find least squares solution to minize gravity misfit for each grav station
        MATDATPAR[i,0:len(df)]=grav_column_der(df_grav.y.iloc[i], # coords of gravity observation points
                                               df_grav.x.iloc[i],
                                               df_grav.z.iloc[i],  
                                               df.northing, df.easting,     
                                               df.top, 
                                               df.bottom,
                                               spacing,     
                                               df.density/1000)  # MATDATPAR[i,0:NBath] is i'th row and columns 0 to NBath

    # Calculate shift to prism's tops to minimize misfit
    # gives the amount that each column's Z1 needs to change by to have the smallest misfit
    Surface_correction=lsqr(MATDATPAR,df_grav[f'inv_misfit_{ITER}'],show=False)[0] # finds the least-squares solution to MATDATPAR and Grav_Misfit, assigns the first value to Surface_correction
    # if necesarry correction is greater than tolerance, then correction equals tolerance, if it's less than tolerance, then correct by Surface_correction
    for i in range(0,len(df)):
        if Surface_correction[i] > max_layer_change_per_iter:
            Surface_correction[i]=max_layer_change_per_iter
        elif Surface_correction[i] < -max_layer_change_per_iter:
            Surface_correction[i]=-max_layer_change_per_iter
    
    # resetting the Z values with the above corrections 
    df.top=df.top+Surface_correction
    
    # Z_Corrected_Bathymetry=df.z[0:len(df)]    
    df['top_diff'] = df.top - layers[active_layer]['df'].z

    # apply the z correction to the prism layer with Harmonica 
    layers[active_layer]['prisms'].prism_layer.update_top_bottom(pygmt.xyz2grd(df[['easting','northing','top']], 
                                region=inv_reg, spacing=spacing), layers[active_layer]['prisms'].bottom)
    
    # recalculate forward gravity of active layer               
    df_grav[f'forward_grav_{active_layer}_{ITER}'] = layers[active_layer]['prisms'].prism_layer.gravity(
    coordinates=(df_grav.x, df_grav.y, df_grav.z),
    field = 'g_z')

    # Calculation of gravity misfit DF, i.e., the difference between calculated and observed gravity
    df_grav[f'forward_grav_total_{ITER}'] = df_grav.forward_grav_total - df_grav[f'{active_layer}_forward_grav'] + df_grav[f'forward_grav_{active_layer}_{ITER}']
    df_grav[f'inv_misfit_{ITER}']=df_grav.grav_inversion-df_grav[f'forward_grav_total_{ITER}']

    print(f"gravity misfit after iteration {ITER} = {int(df_grav[f'inv_misfit_{ITER}'].mean())}")


    # for first iteration, divide infinity by mean square of gravity residuals, inversion will stop once this gets to delta_misfit_squared_tolerance (0.02)
    misfit_sq=np.mean(df_grav[f'inv_misfit_{ITER}']**2)
    delta_misfit_squared=misfit_squared_updated/misfit_sq
    misfit_squared_updated=misfit_sq # updated 
    
    print(f"delta_misfit_squared={delta_misfit_squared}")
    print(f"Iteration {ITER} terminated with least-squares norm={int(misfit_sq)}, executing iteration {ITER+1}")

    # grid and plot the updated bathymetry, difference from starting, and gravity misfit
    layers[active_layer]['inv_grid']=pygmt.xyz2grd(df[['easting','northing','top']], region=inv_reg, spacing=spacing)
    fig = pygmt.Figure()
    
    grid=layers[active_layer]['grid'].z
    fig.grdimage(
            grid,
            cmap='globe',
            projection=inv_proj, 
            region=inv_reg,
            nan_transparent=True,
            frame=['+gwhite'])
    fig.colorbar(cmap=True, frame='af+l"inverted bathymetry (m)"')
    # plot later figure extents (without inv buffer)
    fig.plot(x = [e_fig, e_fig, w_fig, w_fig, e_fig], 
             y = [n_fig, s_fig, s_fig, n_fig, n_fig], pen = '2p,black')
    
    fig.shift_origin(xshift=inv_width/10)
    grid=layers[active_layer]['inv_grid']
    fig.grdimage(
            grid,
            cmap='globe',
            projection=inv_proj, 
            region=inv_reg,
            nan_transparent=True,
            frame=['+gwhite'])
    fig.colorbar(cmap=True, frame='af+l"inverted bathymetry (m)"')
    # plot later figure extents (without inv buffer)
    fig.plot(x = [e_fig, e_fig, w_fig, w_fig, e_fig], 
             y = [n_fig, s_fig, s_fig, n_fig, n_fig], pen = '2p,black')
    
    fig.shift_origin(xshift=inv_width/10)
    diff = pygmt.xyz2grd(df[['easting','northing','top_diff']], region=inv_reg, spacing=spacing)
    try:    
        pygmt.grd2cpt(cmap="polar+h0", grid=diff, region=fig_reg, background=True)
    except:
        print('issue with differences cmap')
        cmap='polar+h0'
    fig.grdimage(
            diff,
            cmap=True,
            projection=inv_proj, 
            region=inv_reg,
            nan_transparent=True,
            frame=['+gwhite'])
    fig.colorbar(cmap=True, frame='af+l"bathymetry difference (m)"')
    # plot later figure extents (without inv buffer)
    fig.plot(x = [e_fig, e_fig, w_fig, w_fig, e_fig], 
             y = [n_fig, s_fig, s_fig, n_fig, n_fig], pen = '2p,black')
    
    fig.shift_origin(xshift=inv_width/10)
    grid = pygmt.xyz2grd(df_grav[['x','y',f'inv_misfit_{ITER}']], region=inv_reg, spacing=grav_spacing)
    pygmt.grd2cpt(cmap="jet", grid=grid, region=fig_reg, background=True)
    fig.grdimage(
            grid,
            cmap=True,
            projection=inv_proj, 
            region=inv_reg,
            nan_transparent=True,
            frame=['+gwhite'])
    fig.colorbar(cmap=True, frame='af+l"gravity misfit (mGal)"')
    # plot later figure extents (without inv buffer)
    fig.plot(x = [e_fig, e_fig, w_fig, w_fig, e_fig], 
             y = [n_fig, s_fig, s_fig, n_fig, n_fig], pen = '2p,black')

    fig.show()

    # stop the inversion if hit the max # iterations or it's below the Least Squares norm 
    if ITER==Max_Iterations:
        print(f"Inversion terminated after {ITER} iterations with least-squares norm={int(misfit_sq)} because maximum number of iterations ({Max_Iterations}) reached")
        break
    if misfit_sq < misfit_sq_tolerance:
        print(f"Inversion terminated after {ITER} iterations with least-squares norm={int(misfit_sq)} because least-squares norm < {misfit_sq_tolerance}")
        break

# end of inversion iteration WHILE loop
if delta_misfit_squared < 1+delta_misfit_squared_tolerance:
    print("terminated - no significant variation in least-squares norm ")

In [ ]:
# surface = xr.load_dataarray("../inversion_layers/BedMachine_surface_5k_filled.nc")
# icebase = xr.load_dataarray("../inversion_layers/BedMachine_icebase_5k_filled.nc")
# bed = xr.load_dataarray("../inversion_layers/BedMachine_bed_5k.nc")
# basement = xr.load_dataarray("../inversion_layers/ROSETTA_basement_BedMachine_bed.nc", drop_variables='polar_stereographic')
# grav = xr.load_dataset('C:/Users/matthewt/Documents/Python_Scripts/RIS_gravity_inversion/inversion_layers/gravity_inv.nc')

# surface = xr.load_dataset('C:/Users/matthewt/Documents/Python_Scripts/RIS_gravity_inversion/inversion_layers/BedMachine_surface_filled_inv.nc')
# icebase = xr.load_dataset('C:/Users/matthewt/Documents/Python_Scripts/RIS_gravity_inversion/inversion_layers/BedMachine_icebase_filled_inv.nc')
# bed = xr.load_dataset('C:/Users/matthewt/Documents/Python_Scripts/RIS_gravity_inversion/inversion_layers/BedMachine_bed_inv.nc')
# basement = xr.load_dataset('C:/Users/matthewt/Documents/Python_Scripts/RIS_gravity_inversion/inversion_layers/ROSETTA_basement_BedMachine_bed_inv.nc')

# df_grav=grav.to_dataframe().reset_index()
# df_grav.rename(columns={'z':'Gobs'}, inplace=True)
# # df_grav['z']=750
# df_grav['z']=surface.to_dataframe().reset_index().z+750
# df_grav

In [ ]:
# TREND_TYPE = 'Constant value'
# # TREND_TYPE = 'Linear'
# # TREND_TYPE = 'Full regional correction'

# # set type of regional correction
# if TREND_TYPE == "Constant value":
#     df_grav['regional_grav'] = df_grav.forward_grav_misfit[(df_grav.x.between(e_fig, w_fig)) & (df_grav.y.between(n_fig, s_fig))].mean()
#     # df_grav['regional_grav'] = df_grav.forward_grav_misfit.mean()
#     print(df_grav['regional_grav'].mean())

# elif TREND_TYPE == 'Linear':
#     # ATEMP=np.column_stack((np.ones(len(df_grav.Gobs[(df_grav.x.between(e_fig, w_fig)) & (df_grav.y.between(n_fig, s_fig))])), 
#     #                         df_grav.y[(df_grav.x.between(e_fig, w_fig)) & (df_grav.y.between(n_fig, s_fig))]))
#     #                         df_grav.x[(df_grav.x.between(e_fig, w_fig)) & (df_grav.y.between(n_fig, s_fig))], 
#     # C,RESID,RANK,SIGMA=np.linalg.lstsq(ATEMP,df_grav.forward_grav_misfit[(df_grav.x.between(e_fig, w_fig)) & (df_grav.y.between(n_fig, s_fig))])
#     # df_grav['regional_grav'] = C[0]*np.ones(len(df_grav.Gobs[(df_grav.x.between(e_fig, w_fig)) & (df_grav.y.between(n_fig, s_fig))])) + C[1]*df_grav.y[(df_grav.x.between(e_fig, w_fig)) & (df_grav.y.between(n_fig, s_fig))] + C[2]*df_grav.x[(df_grav.x.between(e_fig, w_fig)) & (df_grav.y.between(n_fig, s_fig))]
#     ATEMP=np.column_stack((np.ones(len(df_grav.Gobs)), 
#                             df_grav.x, 
#                             df_grav.y))
#     C,RESID,RANK,SIGMA=np.linalg.lstsq(ATEMP,df_grav.forward_grav_misfit)
#     df_grav['regional_grav'] = C[0]*np.ones(len(df_grav.Gobs)) + C[1]*df_grav.y + C[2]*df_grav.x
#     # print(C[0])
    
# elif TREND_TYPE == 'Full regional correction':
#      df_grav['regional_grav'] = df_grav.forward_grav_misfit

# # calculate residual gravity to be inverted
# df_grav['residual_grav'] = df_grav.Gobs - df_grav.regional_grav 

# residual_grav_grid=pygmt.surface(data = df_grav[['x','y','residual_grav']],
#                                  region=inv_reg, 
#                                  spacing=grav_spacing, 
#                                  M='1c', 
#                                  registration='g')

# regional_grav_grid=pygmt.surface(data = df_grav[['x','y','regional_grav']],
#                                  region=inv_reg, 
#                                  spacing=grav_spacing, 
#                                  M='1c', 
#                                  registration='g')

# # plot the grids
# fig = pygmt.Figure()

# # plot observed gravity
# data=grid_grav.z
# pygmt.grd2cpt(cmap="jet", grid=data, region=fig_reg, background=True)
# fig.grdimage(
#         data,
#         cmap=True,
#         projection=fig_proj, 
#         region=fig_reg,
#         nan_transparent=True,
#         frame=['+gwhite'])
# fig.colorbar(cmap=True, frame='af+l"observed gravity (mGal)"')
# # plot later figure extents (without inv buffer)
# fig.plot(x = [fig_reg[0], fig_reg[0], fig_reg[1], fig_reg[1], fig_reg[0]], 
#             y = [fig_reg[2], fig_reg[3], fig_reg[3], fig_reg[2], fig_reg[2]], 
#             pen = '1p,black')

# # plot regional gravity
# fig.shift_origin(xshift=(fig_width)/10)
# data=regional_grav_grid
# try:
#     pygmt.grd2cpt(cmap="jet", grid=data, region=fig_reg, background=True)
#     # pygmt.makecpt(cmap="jet", series=[data.min().item(), data.max().item()], background=True)
# except:
#     pass
# fig.grdimage(
#     data,
#     cmap=True,
#     projection=fig_proj, 
#     region=fig_reg,
#     nan_transparent=True,
#     frame=['+gwhite'])
# fig.colorbar(cmap=True, frame='af+l"regional gravity (mGal)"')
# # plot later figure extents (without inv buffer)
# fig.plot(x = [fig_reg[0], fig_reg[0], fig_reg[1], fig_reg[1], fig_reg[0]], 
#             y = [fig_reg[2], fig_reg[3], fig_reg[3], fig_reg[2], fig_reg[2]], 
#             pen = '1p,black')

# # plot residual gravity
# fig.shift_origin(xshift=(fig_width+1)/10)
# data=residual_grav_grid
# pygmt.grd2cpt(cmap="jet", grid=data, region=fig_reg, background=True)
# fig.grdimage(
#     data,
#     cmap=True,
#     projection=fig_proj, 
#     region=fig_reg,
#     nan_transparent=True,
#     frame=['+gwhite'])
# fig.colorbar(cmap=True, frame='af+l"residual gravity (mGal)"')
# # plot later figure extents (without inv buffer)
# fig.plot(x = [fig_reg[0], fig_reg[0], fig_reg[1], fig_reg[1], fig_reg[0]], 
#             y = [fig_reg[2], fig_reg[3], fig_reg[3], fig_reg[2], fig_reg[2]], 
#             pen = '1p,black')

# fig.show()
# df_grav[(df_grav.x.between(e_fig, w_fig)) & (df_grav.y.between(n_fig, s_fig))].describe()

In [ ]:
# #########
# active_layer = 'layer1'
# #########

# df_grav['grav_inversion']=df_grav.Gobs_shift

# df = layers[active_layer]['prisms'].to_dataframe().reset_index()

# MATDATPAR = (np.zeros([len(df_grav),len(df)])) # array with NG number of rows and NBath+NBase+NM number of columns


# spacing = layers[active_layer]['spacing'] 

# for i,j in enumerate(tqdm(df_grav.Gobs)):
#     # Calculation of the Jacobian matrix using the annular cylinder approximation (grav_column_der)
#     #MATDATPAR is matrix array with NG number of rows and NBath+NBase+NM number of columns
#     # uses vertical derivative of gravity to find least squares solution to minize gravity misfit for each grav station
#     MATDATPAR[i,0:len(df)]=grav_column_der(df_grav.y.iloc[i], # coords of gravity observation points
#                                             df_grav.x.iloc[i],
#                                             df_grav.z.iloc[i],  
#                                             df.northing, df.easting,     
#                                             df.top, 
#                                             df.bottom,
#                                             spacing,     
#                                             df.density/1000)  # MATDATPAR[i,0:NBath] is i'th row and columns 0 to NBath
# MATDATPAR